# Welcome to the Fermionic package

## Why Julia?
I first wrote this code in Mathematica, and was intrigued by performance and usage comparison with Python. There were many advantages of writing the program in Python, like defining classes and methods, which was something very awkward  to emulate in Mathematica. The drawback was, nonetheless, that Python was 10 times slower than Mathematica. This is no surprise, as Mathematica compiles directly in the C language, whereas Python needs to go through numpy in order to access C functionalities (Cython was not an option, as I wanted the source code to be crystal clear to the user). 
Then I heard about Julia. I started learning the language and found it to be extremely  clear to the user, even more than Python. I run the same code and it was 10 times faster than Mathematica, and 100 times faster than Python. Besides, Julia uses something called "multiple dispatch", which replaces the traditional object oriented programming (single dispatch) and is really handy for using the same method with different types of objects. So I decided to try out this language and was more than satisfied with the results.



## Installing Julia and Juno

Lets first cover the setup needed for executing the package.

1. First of all, you must download and install the latest Julia version. You can find it in [this link](https://julialang.org/downloads/). 
2. Once installed, it is necessary to install some interpreter. I strongly recommend using [Juno](https://junolab.org/), which is a package for Atom. You must first download and install [Atom](https://atom.io/). Then from settings, you just type **uber-juno** and wait as many packages are installed. Alternatively, you can download [JuliaPro](https://juliacomputing.com/products/juliapro) that replaces Juno (you will still need Atom).
3. If everything  is working by now, you are ready. If you are working inside Atom, I recommend installing packages directly from the Juno console. You must first type ']' to access "pkg" mode. Then, just type ```add Fermionic``` and the package will be downloaded.
4. If installation was correct, you can start using all the features of the package  by importing it by typing ```using Fermionic ```

In [1]:
using Fermionic

# Operators

We then create elements of the type Op, with dimension n. 
This structure contains the operators as methods. In this package, names change as so.

$a_i \rightarrow a(i)$

$a_i^{\dagger} \rightarrow ad(i)$

$a_ia_j \rightarrow aa(i,j)$

$a_i^{\dagger}a_j^\dagger \rightarrow ada(i,j)$

$a_ia_j^\dagger \rightarrow aad(i,j)$

$a_i^\dagger a_j \rightarrow ada(i)$

$\{a_i,a_j\} = \{a_i^\dagger, a_j^\dagger\} = 0$

$\{a_i,a_j^\dagger\} = \delta_{i,j}$

One of the great advantages of defining these operators as types, is that it is possible to simultaneously define operators for different dimensions. This can be useful for a number of applications, for example for comparing results in different dimensions without re-running the core program each time you switch dimensions.

In Object oriented programming, one would define a class Operator. Here, we are defining a type (like Int or Float) and  a method associated to that kind of type. The big advantage of doing this over defining classes, is that Julia uses **multiple dispatch**. What that means, is that a method can be defined for different input types, and Julia will run the correct method depending on all the input's types. Whereas in object oriented programming, programs select the method from the type of the first input only. This will be very useful for defining states, where we can work both with normal arrays and sparse arrays.

You can do, for instance

op4 = Op(4)

op6 = Op(6)

and op4 will be an element of the type ::Op, and will have all its methods defined for fermionic operators in dimension 4.

In [2]:
op4 = Op(4);

We can see which dimension the operators have by using the dim() method

In [3]:
d = dim(op4)

4

All the operators can be represented by matrices. Let's first look at the basis

In [4]:
basis(op4)

16×4 SparseArrays.SparseMatrixCSC{Float64,Int64} with 32 stored entries:
  [9 , 1]  =  1.0
  [10, 1]  =  1.0
  [11, 1]  =  1.0
  [12, 1]  =  1.0
  [13, 1]  =  1.0
  [14, 1]  =  1.0
  [15, 1]  =  1.0
  [16, 1]  =  1.0
  [5 , 2]  =  1.0
  [6 , 2]  =  1.0
  [7 , 2]  =  1.0
  [8 , 2]  =  1.0
  ⋮
  [8 , 3]  =  1.0
  [11, 3]  =  1.0
  [12, 3]  =  1.0
  [15, 3]  =  1.0
  [16, 3]  =  1.0
  [2 , 4]  =  1.0
  [4 , 4]  =  1.0
  [6 , 4]  =  1.0
  [8 , 4]  =  1.0
  [10, 4]  =  1.0
  [12, 4]  =  1.0
  [14, 4]  =  1.0
  [16, 4]  =  1.0

A sparse matrix is a way of representing a matrix in which one indicates  (row, col) = value. This is really handy for defining matrices with many zeros, as no unnecessary memory is used for storing empty elements. The drawback of the sparse representation is that it's hard to understand what is really doing. We can use the function Matrix() in order to convert a sparse matrix into a normal matrix. I recommend doing this only in order to see what is happening, not for doing operations. This is because both memory and time increase enormously  when operations are done with full matrices instead of sparse.

In [5]:
Matrix(basis(op4))

16×4 Array{Float64,2}:
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  1.0
 0.0  0.0  1.0  0.0
 0.0  0.0  1.0  1.0
 0.0  1.0  0.0  0.0
 0.0  1.0  0.0  1.0
 0.0  1.0  1.0  0.0
 0.0  1.0  1.0  1.0
 1.0  0.0  0.0  0.0
 1.0  0.0  0.0  1.0
 1.0  0.0  1.0  0.0
 1.0  0.0  1.0  1.0
 1.0  1.0  0.0  0.0
 1.0  1.0  0.0  1.0
 1.0  1.0  1.0  0.0
 1.0  1.0  1.0  1.0

We can see that this basis is composed of 16 4-dimensional arrays, each representing a possible  fermionic Slater determinant in 4 dimensions ($2^n$ n-dimensional arrays for dimension n). Each mode can be occupied by 1 or by 0 fermions (due to its antisymmetric  nature). It can be shown, taking the fermionic commutation relations into account, that these states constitute an orthonormal  base. Fermionic operators will be described on this basis. 

For example, let's see the matrix describing $c_i$

In [6]:
Matrix(a(op4,1))

16×16 Array{Float64,2}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     1.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  1.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  1.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  1.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  1.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  1.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  1.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0 

This is the matrix representing the destruction operator corresponding to the mode 1 in 4 dimensions. We can see that when it is applied to the ninth element (1,0,0,0) it takes it to the first element (0,0,0,0). When applied to (1,0,1,1) it goes to (0,0,1,1), and so on. A similar representation can be found for creation operators $c_i^\dagger$

In [7]:
Matrix(ad(op4,4))

16×16 Array{Float64,2}:
 0.0  0.0   0.0  0.0   0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0   0.0  0.0
 1.0  0.0   0.0  0.0   0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0   0.0  0.0
 0.0  0.0   0.0  0.0   0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0   0.0  0.0
 0.0  0.0  -1.0  0.0   0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0   0.0  0.0
 0.0  0.0   0.0  0.0   0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0   0.0  0.0
 0.0  0.0   0.0  0.0  -1.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0   0.0  0.0
 0.0  0.0   0.0  0.0   0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0   0.0  0.0
 0.0  0.0   0.0  0.0   0.0  0.0  1.0     0.0  0.0  0.0  0.0  0.0   0.0  0.0
 0.0  0.0   0.0  0.0   0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0   0.0  0.0
 0.0  0.0   0.0  0.0   0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0   0.0  0.0
 0.0  0.0   0.0  0.0   0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0   0.0  0.0
 0.0  0.0   0.0  0.0   0.0  0.0  0.0     0.0  1.0  0.0  0.0  0.0   0.0  0.0
 0.0  0.0   0.0  0.0   0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0

The minus sign in some operators is a result of the anticommutation relations in fermions.

We can also define some one body operators, such as $c_i^\dagger c_j$:

In [8]:
Matrix(ada(op4,2,3))

16×16 Array{Float64,2}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0 

Of course these compound operators can be also obtained by explicitly computing the product:

In [9]:
ada(op4,2,3) == ad(op4,2)*a(op4,3)

true

When both arguments are equal, we have the number operator (counting occupation of the i mode)

In [10]:
ada(op4,2,2)

16×16 SparseArrays.SparseMatrixCSC{Float64,Int64} with 8 stored entries:
  [5 ,  5]  =  1.0
  [6 ,  6]  =  1.0
  [7 ,  7]  =  1.0
  [8 ,  8]  =  1.0
  [13, 13]  =  1.0
  [14, 14]  =  1.0
  [15, 15]  =  1.0
  [16, 16]  =  1.0

### Unitaries
You can actually do general unitaries of fermionic operatiors by considering the exponential function. This can come in handy for doing any unitary operation, such as a time evolution of a given hamiltonian.

In order to exponentiate a matrix, we must first converte the sparse operator to dense. This has the consequence that we lose the advantages of sparse representations. Therefore this can only be made for relativly small dimensions.

In [11]:
t=1/2
unit = exp(Matrix(im*t*(ada(Op(2),1,2)+ada(Op(2),2,1))))

4×4 Array{Complex{Float64},2}:
 1.0+0.0im       0.0+0.0im            0.0+0.0im       0.0+0.0im
 0.0+0.0im  0.877583+0.0im            0.0+0.479426im  0.0+0.0im
 0.0+0.0im       0.0+0.479426im  0.877583+0.0im       0.0+0.0im
 0.0+0.0im       0.0+0.0im            0.0+0.0im       1.0+0.0im

# States

States are vectors that indicate the coefficient for each element in basis.They can be used both as sparse arrays or as normal arrays. Let's look one more time at the shape of basis:

In [12]:
Matrix(basis(op4))

16×4 Array{Float64,2}:
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  1.0
 0.0  0.0  1.0  0.0
 0.0  0.0  1.0  1.0
 0.0  1.0  0.0  0.0
 0.0  1.0  0.0  1.0
 0.0  1.0  1.0  0.0
 0.0  1.0  1.0  1.0
 1.0  0.0  0.0  0.0
 1.0  0.0  0.0  1.0
 1.0  0.0  1.0  0.0
 1.0  0.0  1.0  1.0
 1.0  1.0  0.0  0.0
 1.0  1.0  0.0  1.0
 1.0  1.0  1.0  0.0
 1.0  1.0  1.0  1.0

We can clearly see that the first element corresponds to a completely empty state. This is the vacuum, and can of course be accessed directly with a method. 
For switching  from sparse to regular vectors,  we now use Array()

In [13]:
vacuum(op4)

16-element SparseArrays.SparseVector{Float64,Int64} with 1 stored entry:
  [1 ]  =  1.0

In [14]:
Array(vacuum(op4))

16-element Array{Float64,1}:
 1.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

Basis has $2^n$ elements. Then a state of the shape 
state = [0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0] corresponds to occupying the only the third mode

In [15]:
ad3 = ad(op4,3);
vac = vacuum(op4)
ad3*vac

16-element SparseArrays.SparseVector{Float64,Int64} with 1 stored entry:
  [3 ]  =  1.0

We can naturally work with more fermions. For example, let's occupy the first three modes:

In [16]:
ad1 = ad(op4,1);
ad2 = ad(op4,2);
ad3 = ad(op4,3);

ad1*ad2*ad3*vac

16-element SparseArrays.SparseVector{Float64,Int64} with 1 stored entry:
  [15]  =  1.0

We can input normal arrays or sparse arrays, both real or complex, to State.

<a id="arrays"></a>
### States

Lets generate a random state of 2 fermions. 

In [17]:
nume = 2
l = length(vac)

random_state = [0.0 for i in 1:l]

for i in 1:l
    if sum(basis(op4)[i,:]) == nume
        random_state[i] = 2*rand(Float64,1)[1]-1
    end
end

print(random_state)

[0.0, 0.0, 0.0, 0.2423922241148917, 0.0, 0.6321678881336128, 0.2750885920516333, 0.0, 0.0, 0.5246032248747636, 0.3709906180609539, 0.0, 0.839767300497857, 0.0, 0.0, 0.0]

We must normalize the state to norm 1.

In [18]:
random_state = random_state/sqrt(random_state'*random_state);
random_state'*random_state

0.9999999999999997

Now we can initialize state as an element of the type State in order to access more information about it. Note that the random_state was defined from a normal array, not a sparse array. We will learn later how to use sparse arrays.

In [19]:
ran_state = State(random_state, op4);

The method belonging to this type are now available:

- st(): prints back the state.
- dim(): returns the corresponding dimension.
- basis(): returns the corresponding basis.
- rhosp(): the one body density matrix, which is the matrix with the one body operator contractions, i.e. $\rho^{\rm sp}(i,j) = \langle \psi | a_j^\dagger a_i |\psi\rangle$.
- rhoqsp(): the one body quasiparticle density matrix, i.e. $\rho^{qsp}$ such that $\rho^{qsp}=\begin{pmatrix}\rho^{sp}&\kappa\\\bar{\kappa}&1-\rho^{sp}\end{pmatrix}$, with $\rho^{sp}$ the one body matrix, $\kappa$ such that $\kappa_{ij}=\langle \psi| a_j a_i|\psi \rangle$.
- n_avg(): the average number of particles.
- eigensp(): the eigenvalues of the rhosp matrix.
- ssp(): the one body entropy which is defined as 
$S(\rho^{\rm sp}) = -\sum_i (\lambda_i \log(\lambda_i) + (1-\lambda_i) \log(1-\lambda_i))$
accounting both for particle ($\lambda_i$) and holes ($1-\lambda_i$).

And many more which can be found in the index.

In [20]:
st(ran_state)

16-element Array{Float64,1}:
 0.0
 0.0
 0.0
 0.1885812279204181
 0.0
 0.49182681924477656
 0.21401901263716677
 0.0
 0.0
 0.4081414768115749
 0.288630819558503
 0.0
 0.6533392284904211
 0.0
 0.0
 0.0

In [21]:
rhosp(ran_state)

4×4 SparseArrays.SparseMatrixCSC{Float64,Int64} with 16 stored entries:
  [1, 1]  =  0.676739
  [2, 1]  =  0.262507
  [3, 1]  =  -0.0628592
  [4, 1]  =  -0.37576
  [1, 2]  =  0.262507
  [2, 2]  =  0.71455
  [3, 2]  =  0.281323
  [4, 2]  =  0.226295
  [1, 3]  =  -0.0628592
  [2, 3]  =  0.281323
  [3, 3]  =  0.164675
  [4, 3]  =  0.223062
  [1, 4]  =  -0.37576
  [2, 4]  =  0.226295
  [3, 4]  =  0.223062
  [4, 4]  =  0.444036

In [22]:
eigensp(ran_state)

4-element Array{Float64,1}:
 0.995271520694479
 0.995271520694478
 0.004728479305521
 0.004728479305521

In [23]:
ssp(ran_state)

0.04333030328828412

We can also define sparse states.

In [24]:
using SparseArrays
d = dim(op4)
l = 2^d

sp_st = spzeros(16)
sp_st[13] = 1

print(typeof(sp_st))
sp_state = State(sp_st, op4);

SparseVector{Float64,Int64}

A random sparse array might be generated in the following way

In [25]:
l= length(vacuum(op4))
nume = 2 #number of particles
state_ran0 = spzeros(l)

for i in 1:l
    if sum(basis(op4)[i,:]) == nume      
        state_ran0[i] = 2*rand(Float64,1)[1]-1
    end
end

state_ran = State(state_ran0, op4);

The most general state is of course complex.

In [26]:
#Random complex array state

op4 = Op(4)

nume = 2
l = length(vac)

state_ran_com = zeros(Complex{Float64},l)

for i in 1:l
    if sum(basis(op4)[i,:]) == nume
        state_ran_com[i] = 2*rand(Complex{Float64},1)[1]-1
    end
end
state_ran_com = state_ran_com/sqrt(state_ran_com'*state_ran_com)
state_ran_com = State(state_ran_com,op4);

In [27]:
Matrix(rhosp(state_ran_com))

4×4 Array{Complex{Float64},2}:
  0.512411+0.0im          0.298208-0.0309074im   …   -0.277161-0.119036im
  0.298208+0.0309074im    0.654759+0.0im            -0.0632625-0.106255im
 -0.237084-0.0789233im    0.338134-0.00168452im       0.207848+0.0853876im
 -0.277161+0.119036im   -0.0632625+0.106255im         0.205566+0.0im

In [28]:
#random complex sparse

using SparseArrays

l= length(vac)
nume = 2 #number of particles
sparse_state_ran_com = spzeros(Complex{Float64},l)

for i in 1:l
    if sum(basis(op4)[i,:]) == nume
        sparse_state_ran_com[i] = 2*rand(Complex{Float64},1)[1]-1
    end
end
sparse_state_ran_com = sparse_state_ran_com/sqrt(sparse_state_ran_com'*sparse_state_ran_com)
sparse_state_ran_com = State(sparse_state_ran_com, op4);

We can check that the rhosp is hermitic

In [29]:
Matrix(rhosp(sparse_state_ran_com))

4×4 Array{Complex{Float64},2}:
     0.3761+0.0im         0.200889-0.0253948im    …  -0.0607094+0.0336499im
   0.200889+0.0253948im   0.362382+0.0im              -0.198253-0.000188833im
   0.234272+0.0943952im   0.148692+0.0600894im         0.179208+0.0374292im
 -0.0607094-0.0336499im  -0.198253+0.000188833im       0.717544+0.0im

Note: in julia the complex conjugation is done with ' applied to a complex matrix. This same application to a real matrix is of course just the transpose.

In [30]:
Matrix(rhosp(sparse_state_ran_com))==Matrix(rhosp(sparse_state_ran_com))'

true

## Some useful states

In [31]:
op4 = Op(4)
ad1 = ad(op4,1)
ad2 = ad(op4,2)
ad3 = ad(op4,3)
ad4 = ad(op4,4)
vac = vacuum(op4)

#initzialize a slater determinant
state_ds0 = ad1*ad2*vac;
state_ds = State(state_ds0, op4) #or just State if you want to work with arrays

#initzialize a maximally entangled state
state_ent = (ad1*ad2 + ad3*ad4)*vac/sqrt(2)
state_ent = State(state_ent, op4); #or just State if you want to work with arrays

#initialization of a random state of 2 particles
using SparseArrays

l= length(vac)
nume = 2 #number of particles

#vectorial real:
state_ran0 = [0.0 for i in 1:l]
for i in 1:l
    if sum(basis(op4)[i,:]) == nume
        state_ran0[i] = 2*rand(Float64,1)[1]-1
    end
end

state_ran0 = state_ran0/sqrt(state_ran0'*state_ran0)
state_ran = State(state_ran0, op4);

#sparse complex
sparse_state_ran_com = spzeros(Complex{Float64},l)

for i in 1:l
    if sum(basis(op4)[i,:]) == nume
        sparse_state_ran_com[i] = 2*rand(Complex{Float64},1)[1]-1
    end
end
sparse_state_ran_com = sparse_state_ran_com/sqrt(sparse_state_ran_com'*sparse_state_ran_com)
sparse_state_ran_com = State(sparse_state_ran_com, op4);